In [ ]:
import json
import pandas as pd
with open("outputs/buckets_07-28_textsim17.json","r") as file:
  buckets = json.load(file)
bucket_map={}
for name in buckets:
  for action in buckets[name]:
    bucket_map[action]=name
bucket_names = list(buckets.keys())
bucket_names_inv={i:name for i,name in enumerate(bucket_names)}
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df_by_term=[pd.read_csv(ds) for ds in datasets]
history_df=pd.concat([pd.read_csv(dataset) for dataset in datasets])
# with open("./outputs/bucket_names_output.jsonl", "r") as file:
#   for line in file:
#     if line.strip():  # Skip empty lines
#       response = json.loads(line)
#       llm_buckets.append(response["response"]["body"]["output"][0]["content"][0]["text"])
      
# history_df["llm_bucket"] = history_df["bucket"].map(dict(zip(bucket_names, llm_buckets)))

# Add next bill_id column to compare
history_df["bucket"]=history_df["action"].apply(lambda action:bucket_map[action])
for h in history_df_by_term:
  h["bucket"]=h["action"].apply(lambda action:bucket_map[action])
bills={bill_id:group for (bill_id,group) in history_df.groupby("bill_id")}
bill_ids = list(bills.keys())

In [ ]:
import numpy as np

alpha=2/3
data=[]

for h in history_df_by_term:
  bill_id=h.iloc[0]["bill_id"]
  predecessors_vec=np.zeroes(len(bucket_names))
  for i,row in h.iterrows():
    predecessor_vec=np.zeroes(len(bucket_names))
    if row["bill_id"]!=bill_id:
      bill_id=row["bill_id"]
      predecessors_vec=np.zeroes(len(bucket_names))
      predecessor_vec=np.zeroes(len(bucket_names))
    one_hot_curr = np.zeroes(len(bucket_names))
    one_hot_curr[bucket_names_inv[row["bucket"]]]=1
    predecessor_vec=(1-alpha)*one_hot_curr+alpha*predecessor_vec
    predecessors_vec[bucket_names_inv[row.action]]+=1
    data.append({"predecessor_vec":predecessor_vec,"predecessors_vec":predecessors_vec,"output":one_hot_curr})
      